In [ ]:
# Copyright 2026 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - ShieldGemma 2 Local Inference


<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_shieldgemma2_local_inference.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_shieldgemma2_local_inference.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates how to install the necessary libraries and run local inference with the ShieldGemma 2 model in a [Colab Enterprise Instance](https://cloud.google.com/colab/docs).

The **ShieldGemma 2** model is trained to detect key harms detailed in the [model card](https://ai.google.dev/gemma/docs/shieldgemma/model_card_2). This guide demonstrates how to use Hugging Face Transformers to build robust data and models.

Note that `ShieldGemma 2` is trained to classify only one harm type at a time, so you will need to make a separate call to `ShieldGemma 2` for each harm type you want to check against. You may have additional that you can use model tuning techniques on `ShieldGemma 2`.

### ShieldGemma 2 Model Licensing

* ShieldGemma 2 Model is available under the [Gemma Terms of Use](https://ai.google.dev/gemma/terms). For full details, refer to the linked documentation that governs the use of this model.


### Objective

* Run local inference with the ShieldGemma 2 model for image safety classification.


### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Supported safety checks

**ShieldGemma2** is a model trained on Gemma 3's 4B IT checkpoint and is trained to detect and predict violations of key harm types listed below:

* **Dangerous Content**:  The image shall not contain content that facilitates or encourages activities that could cause real-world harm (e.g., building firearms and explosive devices, promotion of terrorism, instructions for suicide).

* **Sexually Explicit**: The image shall not contain content that depicts explicit or graphic sexual acts (e.g., pornography, erotic nudity, depictions of rape or sexual assault).

* **Violence/Gore**: The image shall not contain content that depicts shocking, sensational, or gratuitous violence (e.g., excessive blood and gore, gratuitous violence against animals, extreme injury or moment of death).

This serves as a foundation, but users can provide customized safety policies as input to the model, allowing for fine-grained control and specific use-case requirements.

## Before you begin

Make sure you are connecting to a [Colab Enterprise runtime](https://cloud.google.com/colab/docs/connect-to-runtime) with GPU. If not, we recommend [creating a runtime template](https://cloud.google.com/colab/docs/create-runtime-template) with 1 `NVIDIA_L4` GPU (e.g. with the machine type `g2-standard-8` or similar).

In [ ]:
# @title Access the ShieldGemma 2 model

# @markdown 1. Go to the [ShieldGemma 2 model card on Hugging Face](https://huggingface.co/gg-hf-g/shieldgemma-2-4b-it) and accept the agreement if not already.

# @markdown 2. Provide a Hugging Face User Access Token (read) below to access the ShieldGemma 2 model.
# @markdown You can follow the [Hugging Face documentation](https://huggingface.co/docs/hub/en/security-tokens) to create a **read** access token.

HF_TOKEN = ""  # @param {type:"string", isTemplate:true}

In [ ]:
# @title Install transformers with ShieldGemma 2 support

! pip install git+https://github.com/huggingface/transformers@v4.50.0

## Image Safety Classification with ShieldGemma 2

The following code classifies images for safety with the ShieldGemma 2 model.

**Input:**

Image + Prompt Instruction with policy definition.

**Output:**

Probability of `Yes`/`No` tokens, with a higher score indicating the model's higher confidence that the image violates the specified policy. `Yes` means that the image violated the policy, `No` means that the model did not violate the policy.


In [ ]:
# @title Load model weights
import torch
from transformers import AutoProcessor, ShieldGemma2ForImageClassification

model_id = "google/shieldgemma-2-4b-it"

processor = AutoProcessor.from_pretrained(model_id, token=HF_TOKEN)
model = ShieldGemma2ForImageClassification.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    token=HF_TOKEN,
)
model = model.to(torch.device("cuda"))

In [ ]:
import requests
# @title Run local inference with input image
from PIL import Image

# @markdown URL to the input image.
image_url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG"  # @param {type: "string"}
# @markdown Click "Show Code" to see the more details.

image = Image.open(requests.get(image_url, stream=True).raw)
model_inputs = processor(images=[image], return_tensors="pt").to(model.device)

with torch.inference_mode():
    scores = model(**model_inputs)

print(scores.probabilities)